In [3]:
import pandas as pd 
import numpy as np
from dash_website import DIMENSIONS
from dash_website.utils.graphs.dendrogram_heatmap import create_dendrogram_heatmap

main_category = "Biomarkers"
category = "Alcohol"
correlation_type = "spearman"

data = pd.read_feather("../../data/xwas/univariate_correlations/averages_correlations_all.feather").to_dict()
correlations = pd.DataFrame(data).set_index(["dimension_1", "dimension_2", "category"])
correlations.columns = pd.MultiIndex.from_tuples(
    list(map(eval, correlations.columns.tolist())), names=["category", "variable"]
)

if category == "All":
    category = f"All_{main_category}"

matrix_correlations_2d = correlations[correlation_type].swaplevel().swaplevel(i=0, j=1).loc[category, "mean"]
matrix_correlations_2d.name = "correlation"
matrix_correlations_2d_trim = matrix_correlations_2d.loc[DIMENSIONS]
correlations_2d = pd.pivot_table(
    matrix_correlations_2d_trim.to_frame(), values="correlation", index="dimension_1", columns="dimension_2", dropna=False
).fillna(0)
correlations_2d[DIMENSIONS[0]] = 0  # add correlations of 0 since upper matrix

correlations_values = correlations_2d.loc[DIMENSIONS, DIMENSIONS].values
correlations_values[np.tril_indices(correlations_values.shape[0], -1)] = correlations_values.T[np.tril_indices(correlations_values.shape[0], -1)]
correlations_2d.loc[DIMENSIONS, DIMENSIONS] = correlations_values


matrix_number_variables_2d = correlations["number_variables"].swaplevel().swaplevel(i=0, j=1).loc[category, "number_variables"]
matrix_number_variables_2d.name = "number_variables"
matrix_number_variables_2d_trim = matrix_number_variables_2d.loc[DIMENSIONS]
number_variables_2d = pd.pivot_table(
    matrix_number_variables_2d_trim.to_frame(), values="number_variables", index="dimension_1", columns="dimension_2", dropna=False
).fillna(0)
number_variables_2d[DIMENSIONS[0]] = 0  # since upper matrix

number_variables_values = number_variables_2d.loc[DIMENSIONS, DIMENSIONS].values
number_variables_values[np.tril_indices(number_variables_values.shape[0], -1)] = number_variables_values.T[np.tril_indices(number_variables_values.shape[0], -1)]
number_variables_2d.loc[DIMENSIONS, DIMENSIONS] = number_variables_values

ipykernel_launcher:19: PerformanceWarning: indexing past lexsort depth may impact performance.
ipykernel_launcher:32: PerformanceWarning: indexing past lexsort depth may impact performance.


In [4]:
fig = create_dendrogram_heatmap(
correlations_2d.loc[DIMENSIONS, DIMENSIONS], 
number_variables_2d.loc[DIMENSIONS, DIMENSIONS])

fig.show()

In [31]:
len(pd.Index(DIMENSIONS).drop(dimension))

32

In [3]:
import pandas as pd 
import numpy as np
from dash_website import DIMENSIONS
from dash_website.utils.graphs.dendrogram_heatmap import create_dendrogram_heatmap

dimension = "BrainCognitive"
correlation_type = "spearman"

data = pd.read_feather("../../data/xwas/univariate_correlations/averages_correlations_all.feather").to_dict()
correlations = pd.DataFrame(data).set_index(["dimension_1", "dimension_2", "category"])
correlations.columns = pd.MultiIndex.from_tuples(
    list(map(eval, correlations.columns.tolist())), names=["category", "variable"]
)


matrix_correlations_2d_1 = correlations[correlation_type].loc[dimension, "mean"].swaplevel()
matrix_correlations_2d_1.name = "correlation"
correlations_2d_1 = pd.pivot_table(
    matrix_correlations_2d_1.to_frame(), values="correlation", index="dimension_2", columns="category",dropna=False
).fillna(0)

matrix_correlations_2d_2 = correlations[correlation_type].swaplevel(i=0, j=1).loc[dimension, "mean"]
matrix_correlations_2d_2.name = "correlation"
correlations_2d_2 = pd.pivot_table(
    matrix_correlations_2d_2.to_frame(), values="correlation", index="dimension_1", columns="category",dropna=False
).fillna(0)
correlations_2d_concat = pd.concat([correlations_2d_1, correlations_2d_2])
correlations_2d = correlations_2d_concat.loc[pd.Index(DIMENSIONS).drop(dimension)]  # remove dimension since upper matrix


matrix_number_variables_2d_1 = correlations["number_variables"].loc[dimension, "number_variables"].swaplevel()
matrix_number_variables_2d_1.name = "number_variables"
number_variables_2d_1 = pd.pivot_table(
    matrix_number_variables_2d_1.to_frame(), values="number_variables", index="dimension_2", columns="category", dropna=False
).fillna(0)

matrix_number_variables_2d_2 = correlations["number_variables"].swaplevel(i=0, j=1).loc[dimension, "number_variables"]
matrix_number_variables_2d_2.name = "number_variables"
number_variables_2d_2 = pd.pivot_table(
    matrix_number_variables_2d_2.to_frame(), values="number_variables", index="dimension_1", columns="category", dropna=False
).fillna(0)
number_variables_2d_concat = pd.concat([number_variables_2d_1, number_variables_2d_2]).astype(int)
number_variables_2d = number_variables_2d_concat.loc[pd.Index(DIMENSIONS).drop(dimension)]  # remove dimension since upper matrix

ipykernel_launcher:16: PerformanceWarning: indexing past lexsort depth may impact performance.
ipykernel_launcher:22: PerformanceWarning: indexing past lexsort depth may impact performance.
ipykernel_launcher:31: PerformanceWarning: indexing past lexsort depth may impact performance.
ipykernel_launcher:37: PerformanceWarning: indexing past lexsort depth may impact performance.


In [6]:
import plotly.graph_objs as go
from dash_website.utils.graphs.colorscale import get_colorscale

hovertemplate = "Correlation: %{z:.3f} <br>X subcategory: %{x} <br>Aging dimension: %{y} <br>Number variables: %{customdata} <br><extra></extra>"

heatmap = go.Heatmap(
    x=correlations_2d.columns,
    y=correlations_2d.index,
    z=correlations_2d,
    colorscale=get_colorscale(correlations_2d),
    customdata=number_variables_2d,
    hovertemplate=hovertemplate,
)

fig = go.Figure(heatmap)

fig.update_layout(
    {
        "width": 2000,
        "height": 1000,
        "xaxis": {"title": "X subcategory", "tickangle": 90, "showgrid": False},
        "yaxis": {"title": "Aging dimension", "showgrid": False},
    }
)